In [3]:
import requests
import datetime
import os

# API key
api_key = os.getenv("OPENAI_API_KEY")

# API headers
headers = {'Authorization': f'Bearer {api_key}'}

# API endpoint
url = 'https://api.openai.com/v1/usage'

# Date for which to get usage data
date = datetime.date(2023, 11, 12)

# Parameters for API request
params = {'date': date.strftime('%Y-%m-%d')}

# Send API request and get response
response = requests.get(url, headers=headers, params=params)
usage_data = response.json()['data']

In [4]:
calc_total_tokens = lambda item: item['n_context_tokens_total'] + item['n_generated_tokens_total']

accumulated_data = {}

# There are often duplicate snapshot_id, so merge them
for item in usage_data:
    if calc_total_tokens(item) > 0:
        snapshot_id = item['snapshot_id']

        if snapshot_id not in accumulated_data:
            # Initialize a new entry
            accumulated_data[snapshot_id] = item.copy()
            accumulated_data[snapshot_id]['n_tokens_total'] = calc_total_tokens(item)
        else:
            # Update the existing entry by summing up the values
            accumulated_data[snapshot_id]['n_requests'] += item['n_requests']
            accumulated_data[snapshot_id]['n_context_tokens_total'] += item['n_context_tokens_total']
            accumulated_data[snapshot_id]['n_generated_tokens_total'] += item['n_generated_tokens_total']
            accumulated_data[snapshot_id]['n_tokens_total'] += calc_total_tokens(item)

filtered_data = list(accumulated_data.values())

# Print table headers
print(f"{'Snapshot ID':30} {'Requests':10} {'Context Tokens':20} {'Generated Tokens':20} {'Total Tokens':20}")

# Iterate and print each item in a formatted way
for item in filtered_data:
    print(f"{item['snapshot_id']:30} {item['n_requests']:10} {item['n_context_tokens_total']:20} {item['n_generated_tokens_total']:20} {item['n_tokens_total']:20}")


Snapshot ID                    Requests   Context Tokens       Generated Tokens     Total Tokens        
gpt-4-0613                              4                 3665                  388                 4053
gpt-4-1106-vision-preview               8                 9005                  128                 9133
